In [1]:
#importing libraries
import pandas as pd
#Model
from sklearn.metrics.pairwise import cosine_similarity
#Deployment
import pickle
import streamlit as st

In [2]:
# Function for popularity-based recommendations
def popularity_recommendation_with_titles(df, n_recommendations=5):
    # Group by ISBN and count the number of ratings
    book_popularity = df.groupby('ISBN')['book_rating'].count().reset_index(name='popularity')

    # Sort books by popularity in descending order
    sorted_books = book_popularity.sort_values(by='popularity', ascending=False)

    # Get the top N recommendations
    top_recommendations = sorted_books.head(n_recommendations)

    # Extract ISBNs from the recommendations
    recommendations_isbn = top_recommendations['ISBN'].tolist()

    # Extract book titles using the ISBN codes
    recommendations_titles = [
        df[df['ISBN'] == isbn]['booktiltle'].iloc[0] for isbn in recommendations_isbn
    ]

    return recommendations_titles

In [3]:
# Read the full DataFrame
df = pd.read_csv('Combined_Dataframes.csv')

In [4]:
# Downsize the DataFrame using popularity-based recommendations
downsized_books = popularity_recommendation_with_titles(df, n_recommendations=3000)  # Choose an appropriate number

In [5]:
# Create a new DataFrame based on the downsized book list
new_df = df[df['booktiltle'].isin(downsized_books)]

In [6]:
# Calculate cosine similarity on the new DataFrame
book_reader_matrix = pd.get_dummies(new_df[['bookauthor', 'Location']])
book_reader_similarity = cosine_similarity(book_reader_matrix)

MemoryError: Unable to allocate 10.9 GiB for an array with shape (118484, 12374) and data type float64

In [ ]:
# Streamlit app begins
st.title('Book Recommendation App')
user_input_id = st.text_input("Enter the userID: ")

In [ ]:
if user_input_id:
    user_input_id = int(user_input_id)

    # Check if the entered user ID exists in the DataFrame
    if user_input_id not in new_df['userID'].values:
        st.write('Invalid UserID')
    else:
        st.write('UserID: ', user_input_id)

        # Retrieve the index of the user
        user_index = new_df[new_df['userID'] == user_input_id].index[0]

        # Use cosine similarity on the new DataFrame for recommendations
        similarity_scores = book_reader_similarity[user_index]

        # Sort and get top similar users (excluding the user itself)
        similar_users_indices = similarity_scores.argsort()[::-1][1:]

        # Recommend books based on most similar user
        most_similar_user_index = similar_users_indices[0]
        recommended_books_indices = book_reader_similarity[most_similar_user_index].argsort()[::-1]
        recommended